In [13]:
import numpy as np
import json
import DnnLib

MODEL_PATH = "../modelos/testmodel.json"
TEST_DATA_PATH = "../data/fashion_test.npz"

data_test = np.load(TEST_DATA_PATH)
images_test = data_test["images"].astype(np.float64)
labels_test = data_test["labels"]

X_test = images_test.reshape(len(images_test), -1) / 255.0
y_test = np.eye(10)[labels_test]  # one-hot encoding

print("Datos de test cargados.")

# -----------------------------
# Función para cargar modelo
# -----------------------------
def load_model(filename):
    with open(filename, "r") as f:
        model_data = json.load(f)

    layers = []
    for layer_info in model_data:
        if layer_info["type"] == "DenseLayer":
            layer = DnnLib.DenseLayer(
                layer_info["input_dim"],
                layer_info["output_dim"],
                getattr(DnnLib.ActivationType, layer_info["activation"])
            )
            layer.weights[:] = np.array(layer_info["weights"], dtype=np.float64)
            layer.bias[:] = np.array(layer_info["bias"], dtype=np.float64)

            # Reiniciar cualquier estado interno
            if hasattr(layer, 'cache'):
                layer.cache = None
            if hasattr(layer, 'grad_cache'):
                layer.grad_cache = None

            layers.append(layer)
    print(f"Modelo cargado desde {filename}")
    return layers

layers = load_model(MODEL_PATH)

preds = X_test
for layer in layers:
    preds = layer.forward(preds)

# -----------------------------
# Calcular accuracy
# -----------------------------
pred_classes = np.argmax(preds, axis=1)
true_classes = np.argmax(y_test, axis=1)
accuracy = np.mean(pred_classes == true_classes)

print(f"Accuracy en test: {accuracy:.4f}")


Datos de test cargados.
Modelo cargado desde ../modelos/testmodel.json
Accuracy en test: 0.0778
